<a href="https://colab.research.google.com/github/ViniciusSiqueira54/InteractiveTestBook/blob/master/Data_Analysis/Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Analysis

File used to analysis the data related to all plans of one device, with these data we'll begin to predict which data will be usefull to construct our heuristic for the creation of new plans throw *History Based Priorisation*.

In [1]:
import pandas as pd
import matplotlib
import ipywidgets as widgets

#Useful functions
# data.columns.values # (Return an array representing the data in the Index.)
# data.head() # (Return the first n rows. This function returns the first n rows for the object based on position. 
# It is useful for quickly testing if your object has the right type of data in it.)



Import the Unified Data Base with all Platform Regression Test Plans data information related to a single device, and also the Data Base with all the CRs used on these Regression Plans for this device.

Note: Every time that a new commit was made by the File Processor, the two links below needs to be updated, because the token for these files will be changed as well.

In [ ]:
url_data = 'https://raw.githubusercontent.com/mirandaRG/hbprio/master/Data_Source/Voyager_AndroidP/00_UnifiedData_Master.csv?token=AO47UI2KYJB37TDPV4BJGAS7SXMEI'
data = pd.read_csv(url_data, sep=",",encoding="UTF-8")

url_CRs_data = 'https://raw.githubusercontent.com/mirandaRG/hbprio/master/Data_Source/Voyager_AndroidP/CRs_All_Plans.csv?token=AO47UIYVPZZQNLSHM3UXSUK7SXMEM'
crs_data = pd.read_csv(url_CRs_data, sep=",",encoding="UTF-8")

Return the number of cases related to the Test Result type:

In [ ]:
columnTestResult = data['TestResults']
columnTestResult.value_counts()

Choose the Test Plnas by the category of the plan, using its campaign label.

In [ ]:
innerData = pd.DataFrame()
planData = pd.DataFrame()
chooseLabel = ""

In [ ]:
dropdown_campaign = widgets.Dropdown(options = data.CampaignLabel.dropna().unique())
display(dropdown_campaign)

out1 = widgets.Output()
display(out1)

def on_change_campaign(change):
  global chooseLabel
  with out1:
    if change['type'] == 'change' and change['name'] == 'value':
        display(displayTPsIds(change['new']))
        chooseLabel = change['new']

dropdown_campaign.observe(on_change_campaign)

def displayTPsIds (campaign):
  global innerData
  out1.clear_output()
  innerData = data.filter(items=['CampaignLabel','TestPlanID']).where(data.CampaignLabel == campaign).dropna().drop_duplicates()
  return innerData

In [ ]:
dropdown_testPlan = widgets.Dropdown(options = data.TestPlanID.where(data.CampaignLabel == chooseLabel).dropna().unique())
display(dropdown_testPlan)

out2 = widgets.Output()
out3 = widgets.Output()

display(out3)
display(out2)

def on_change_testPlan(change):
  with out3:
    if change['type'] == 'change' and change['name'] == 'value':
      display(planHistory(change['new']).value_counts())
  with out2:
    if change['type'] == 'change' and change['name'] == 'value':
      display(choosePlan(change['new']))

#Observer
dropdown_testPlan.observe(on_change_testPlan)


#Methods
def choosePlan (TPid):
  global planData
  out2.clear_output()

  planData = data.filter(items=['TestCaseID','TestResults','AffectsVersion','BlockedCRNumber','BlockedReason','RemoteDefectCR']).where(data.TestPlanID == TPid).dropna(how='all').drop_duplicates()

  auxFail = planData.where(planData.TestResults == 'Fail').dropna(how='all')
  auxBlocked = planData.where(planData.TestResults == 'Blocked').dropna(how='all')

  mergeData = [auxFail,auxBlocked]
  allCrs = pd.concat(mergeData, ignore_index=True, sort=False)
    
  return allCrs  

def planHistory(TPid):
  global planData
  out3.clear_output()

  planData = data.filter(items=['TestCaseID','TestResults','AffectsVersion','BlockedCRNumber','BlockedReason','RemoteDefectCR']).where(data.TestPlanID == TPid).dropna(how='all').drop_duplicates()
  columnTestResult = planData['TestResults']
  #columnTestResult.value_counts()

  return columnTestResult 

In [ ]:
testDiff01 = pd.DataFrame()
testDiff02 = pd.DataFrame()


testDiff01 = data.filter(items=['TestCaseID','TestResults']).where(data.TestPlanID == "MCA-1872700").dropna(how='all').drop_duplicates() # MCA-1872700 Core Reg Cycle 1
testDiff02 = data.filter(items=['TestCaseID','TestResults']).where(data.TestPlanID == "MCA-2053888").dropna(how='all').drop_duplicates() # MCA-2053888 Core Reg Cycle 2

In [ ]:
df = testDiff01.merge(testDiff02)
df

In [ ]:
df = testDiff02.merge(testDiff01, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']

df

# Trials

In [ ]:
filteredData = data.filter(items=['TestPlanID','Issuekey','AffectsVersion','BlockedCRNumber','BlockedReason','CampaignLabel','RemoteDefectCR','TestResults'])

In [ ]:
failCases = filteredData.filter(items=['Issuekey', 'IssueID', 'AffectsVersion', 'RemoteDefectCR', 'TestResults'])\
.where(filteredData.TestResults == 'Fail')\
.groupby('Issuekey')

failCases.head().dropna(how='all')

In [ ]:
blockedCases = filteredData.filter(items=['Issuekey', 'IssueID', 'AffectsVersion', 'BlockedCRNumber', 'BlockedReason', 'TestResults'])\
.where(filteredData.TestResults == 'Blocked')\
.groupby('Issuekey')

blockedCases.head().dropna(how='all')

In [ ]:
data.filter(items=['TestResults','RemoteDefectCR']).dropna().drop_duplicates()

In [ ]:
data.filter(items=['TestResults','BlockedCRNumber','BlockedReason']).where(data.TestResults == 'Blocked').dropna(how='all').drop_duplicates()

In [ ]:
data.filter(items=['TestResults','BlockedReason']).where(data.TestResults == 'Blocked').dropna(how='all').drop_duplicates()

In [ ]:
data.filter(items=['TestResults','BlockedReason']).where(data.TestResults == 'Blocked').dropna(how='all').groupby('BlockedReason').count()

In [ ]:
dt_blocked = data.filter(items=['TestResults','BlockedReason']).where(data.TestResults == 'Blocked').dropna().groupby('BlockedReason').count()

In [ ]:
data.filter(items=['CampaignLabel']).dropna().drop_duplicates()

In [ ]:
print(chooseLabel)

In [ ]:
data.filter(items=['CampaignLabel','TestPlanID']).dropna().drop_duplicates()

In [ ]:
data.filter(items=['CampaignLabel','TestPlanID']).where(data.CampaignLabel == 'olson_row_p_core_reg').dropna().drop_duplicates()

In [ ]:
filteredData.head()

In [ ]:
columnTestResult = filteredData['TestResults']
columnTestResult.value_counts()

In [ ]:
SelectedByCategory = filteredData.where(filteredData.CampaignLabel == 'olson_row_p_core_reg').dropna(how='all')


In [ ]:
SelectedByCategory.head()

In [ ]:
fail = columnTestResult.where(SelectedByCategory.TestResults == 'Fail').value_counts()
blocked = columnTestResult.where(SelectedByCategory.TestResults == 'Blocked').value_counts()

print(fail)
print(blocked)

In [ ]:
SelectedByCategory.where(filteredData.TestPlanID == 'MCA-2046169').dropna(how='all')

In [ ]:
SelectedByCategory.where(filteredData.TestPlanID == 'MCA-2046169').dropna(how='all').value_counts('TestResults')

In [ ]:
selected = SelectedByCategory.where(filteredData.TestPlanID == 'MCA-2046169').dropna(how='all')

In [ ]:
auxFail = selected.where(filteredData.TestResults == 'Fail').dropna(how='all')
auxBlocked = selected.where(filteredData.TestResults == 'Blocked').dropna(how='all')

mergeData = [auxFail,auxBlocked]
allCrs = pd.concat(mergeData, ignore_index=True, sort=False)
allCrs.head()

CRs Data

In [ ]:
crs_data.head()

#APFD

In [19]:
url_data = 'https://raw.githubusercontent.com/mirandaRG/hbprio/master/Data_Source/Voyager_AndroidP/MCA-1872700_processed.csv?token=AO47UI27ALRAAGPMPCMTRHLAXQCB4'
apfd_data = pd.read_csv(url_data, sep=",",encoding="UTF-8")

apfd_filtered = apfd_data.filter(items=['TestCaseID','Summary','AffectsVersion','TestResults','RemoteDefectCR','BlockedCRNumber','BlockedReason','TestDate'])

In [20]:
# pd.to_datetime(df)
# convert the 'Date' column to datetime format
#df['Date']= pd.to_datetime(df['Date'])

apfd_filtered['TestDate'] = pd.to_datetime(apfd_filtered['TestDate'])

# df_ordenado = df.sort_values(by='Data')

apfd_ordered = apfd_filtered.sort_values(by='TestDate')

# cria uma nova coluna
apfd_ordered['ExecOrd'] = 'NaN'
count = 0

for index, row in apfd_ordered.iterrows():
  count += 1
  apfd_ordered.loc[index,'ExecOrd'] = count


In [21]:
apfd_ordered.head()

,TestCaseID,Summary,AffectsVersion,TestResults,RemoteDefectCR,BlockedCRNumber,BlockedReason,TestDate,ExecOrd
451,MCA-198225,TE - TC - FPS Out of Box Setup,PPV29.257,Pass,NaN,NaN,NaN,2019-08-26 08:26:00,1
440,MCA-706877,"TE - TC - TC - Unlock via wrong fingerprint, t...",PPV29.257,Pass,NaN,NaN,NaN,2019-08-26 08:29:00,2
442,MCA-24801,TE - TC - AFW Managed Account - Gmail check in...,PPV29.257,Pass,NaN,NaN,NaN,2019-08-26 08:38:00,3
441,MCA-706876,TE - TC - TC - The DUT power off during record...,PPV29.257,Pass,NaN,NaN,NaN,2019-08-26 08:40:00,4
462,MCA-1604036,TE - TC - Fingerprint - Simulate the pocket mode,PPV29.257,Pass,NaN,NaN,NaN,2019-08-26 08:46:00,5
